In [ ]:
# Version 1 This is just the base program
import pygame
import numpy as np
import sys
import random

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50
PERSON_RADIUS = 5
MAX_SPEED = 2
SEPARATION_RADIUS = 25
WALL_MARGIN = 10

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
    
    def follow_path(self, path_waypoints):
        if self.path_index < len(path_waypoints):
            target = path_waypoints[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index = (self.path_index + 1) % len(path_waypoints)  # Loop path
            else:
                # Normalize direction and apply to velocity
                if distance > 0:
                    direction_normalized = direction / distance
                    self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        # Follow the path
        self.follow_path(path_waypoints)
        
        # Separation from other people
        separation = np.zeros(2)
        # Se itera entre todas las instancias menos la que se está analizando
        for other in people:
            if other != self:
                dist = np.linalg.norm(self.position - other.position) # Se calcula la distancia
                if dist < SEPARATION_RADIUS: # Si la distancia de separación es menor que el límite permitido, se aleja
                    separation += (self.position - other.position) / (dist + 0.0001)  # Avoid division by zero
        
        # Wall avoidance (only for solid walls)
        # Se itera entre todas las paredes
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            # Si la distancia es menor a la permitida se separa de la pared.
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        # Pass through passable walls (no avoidance)
        # We could add special behavior here if needed
        
        # Update velocity and position
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        
        # Keep within screen bounds
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        pygame.draw.circle(screen, RED, self.position.astype(int), self.radius)

# Define path waypoints (green)
path_waypoints = [# A vector is define like this [x1, y1]
    np.array([200, 100]),
    np.array([1600, 200]),
    np.array([1600, 800]),
    np.array([200, 800])
]

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [0, 400, 800, 400],      # Horizontal wall left
    [1000, 400, WIDTH - 150, 400], # Horizontal wall right (gap between 800-1000)
    
    #[900, 400, 900, 0],      # Vertical wall top
    [900, 600, 900, HEIGHT]  # Vertical wall bottom (gap between 400-600)
]

# Define passable walls (blue - can walk through)
passable_walls = [
#    [800, 400, 1000, 400],   # Passage between horizontal walls
#    [900, 400, 900, 600]     # Passage between vertical walls
]

# Create people
people = [Person(random.randint(0, WIDTH), random.randint(0, HEIGHT)) 
          for _ in range(NUM_PEOPLE)]

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, BLUE, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

In [ ]:
# Version 1.2 This is the program with only the interval timing
import pygame
import numpy as np
import sys
import random
from pygame.locals import *

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 100
PERSON_RADIUS = 5
MAX_SPEED = 2
SEPARATION_RADIUS = 20
WALL_MARGIN = 10
SPAWN_AREA = (20, 150, 20, 100)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 10                   # Number of people per group
SPAWN_INTERVAL = 2000            # Milliseconds between group spawns

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255, 128)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

class Person:
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20
        self.active = True
    
    def is_in_passage(self, position):
        for wall in passable_walls:
            x1, y1, x2, y2 = wall
            if (min(x1,x2)) <= position[0] <= max(x1,x2) and (min(y1,y2)) <= position[1] <= max(y1,y2):
                return True
        return False
    
    def follow_path(self, path_waypoints):
        if self.path_index < len(path_waypoints):
            target = path_waypoints[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index = (self.path_index + 1) % len(path_waypoints)
            elif distance > 0:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        if not self.active:
            return
            
        self.follow_path(path_waypoints)
        
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.001)
        
        if not self.is_in_passage(self.position):
            for wall in walls:
                dist_to_wall = self.distance_to_wall(wall)
                if dist_to_wall < WALL_MARGIN:
                    separation += (self.position - wall[:2]) / (dist_to_wall + 0.001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if self.active:
            pygame.draw.circle(screen, RED, self.position.astype(int), self.radius)

# Initialize empty people list
people = []
active_count = 0

# Path waypoints
path_waypoints = [
    np.array([200, 200]),
    np.array([1600, 200]),
    np.array([1600, 800]),
    np.array([200, 800])
]

# Walls
walls = [
    [0, 0, WIDTH, 0],
    [WIDTH, 0, WIDTH, HEIGHT],
    [WIDTH, HEIGHT, 0, HEIGHT],
    [0, HEIGHT, 0, 0],
    [0, 400, 800, 400],
    [1000, 400, WIDTH, 400],
    [900, 400, 900, 0],
    [900, 600, 900, HEIGHT]
]

# Passable walls
passable_walls = [
    [800, 380, 1000, 420],
    [890, 400, 910, 600]
]

# Spawn timer
last_spawn_time = 0

running = True
while running:
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
    
    # Spawn new groups periodically
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Spawn a new group
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            if len(people) < NUM_PEOPLE:
                # Create new person
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                people.append(Person(x, y))
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        person.position = np.array([
                            random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                            random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        ], dtype=float)
                        person.velocity = np.random.rand(2) * MAX_SPEED
                        person.active = True
                        break
        active_count += spawn_count
        last_spawn_time = current_time
    
    screen.fill(WHITE)
    
    # Draw path
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), path_waypoints[i-1], point, 2)
    
    # Draw solid walls
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls
    for wall in passable_walls:
        x1, y1, x2, y2 = wall
        rect = pygame.Rect(min(x1,x2), min(y1,y2), abs(x2-x1), abs(y2-y1))
        s = pygame.Surface((abs(x2-x1), abs(y2-y1)), pygame.SRCALPHA)
        s.fill(BLUE)
        screen.blit(s, (min(x1,x2), min(y1,y2)))
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
        if person.active:
            active_count += 1
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

In [1]:
# Version 1.3

# This is just the program with a supermarket simulation
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50
PERSON_RADIUS = 5
MAX_SPEED = 1
SEPARATION_RADIUS = 20
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 150, 30, 80)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 2                   # Number of people per group
SPAWN_INTERVAL = 3500            # Milliseconds between group spawns
INFECTION_RATE = 0.2  # 20% de las personas estarán infectadas

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [np.array([200, 100])]


# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [np.array([350, 100]),
                        np.array([50, 100]),]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    #np.array([1625, 100]),
    #np.array([275, 100]),
    
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),
    
    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),
    
    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),
    
    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),
    
    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
    
    np.array([600, 700]),
    np.array([850, 700]),
    np.array([1100, 700]),
    np.array([1350, 700]),

]

# Cajeros
#middle_path_waypoints = [np.array([1600, 800]),
#                          np.array([300, 800])]

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True
        self.path = path  # Each person gets its own path
        self.color = BLUE # Susceptible person
        
    
    def follow_path(self):
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            if distance < self.path_threshold:
                self.path_index += 1
                # If reached final waypoint, deactivate
                if self.path_index >= len(self.path):
                    self.active = False
            elif distance > 0:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1
    
    def update(self, people, walls, passable_walls):
        # Si no se encuentra activo la persona, que se devuelva
        if not self.active:
            return
        
        # Follow the path
        self.follow_path()
        
        # Vector para guardar la separación entre otras personas
        separation = np.zeros(2)
        # Se itera entre todas las instancias menos la que se está analizando
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position) # Se calcula la distancia
                if dist < SEPARATION_RADIUS: # Si la distancia de separación es menor que el límite permitido, se aleja
                    separation += (self.position - other.position) / (dist + 0.0001)  # Avoid division by zero
        
        # Wall avoidance (only for solid walls)
        # Se itera entre todas las paredes
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            # Si la distancia es menor a la permitida se separa de la pared.
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        # Pass through passable walls (no avoidance)
        # We could add special behavior here if needed
        
        # Update velocity and position
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        
        # Keep within screen bounds
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if self.active:
            pygame.draw.circle(screen, self.color, self.position.astype(int), self.radius)

        

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1700, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
    [400, 650, 1500, 650], # division 6
    
    
    # Original walls
    #[0, 400, 800, 400],      # Horizontal wall left
    #[1000, 400, WIDTH - 150, 400], # Horizontal wall right (gap between 800-1000)
    
    #[900, 400, 900, 0],      # Vertical wall top
    #[900, 600, 900, HEIGHT]  # Vertical wall bottom (gap between 400-600)
]

# Define passable walls (blue - can walk through)
passable_walls = [
#    [800, 400, 1000, 400],   # Passage between horizontal walls
#    [900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(3, 8)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points
    full_path = selected_waypoints + final_path_waypoints
    
    # We intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            if len(people) < NUM_PEOPLE:
                # Create new person
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path)
                people.append(new_person)
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        person.position = np.array([
                            random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                            random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        ], dtype=float)
                        person.velocity = np.random.rand(2) * MAX_SPEED
                        person.path = group_path
                        person.path_index = 0
                        person.active = True
                        break
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(middle_path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           middle_path_waypoints[i-1], point, 2)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls, passable_walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/home/nelson/Desktop/master/tfm/code/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
